In [2]:
import requests

from bs4 import BeautifulSoup as bs

import pandas as pd

import numpy as np

import time

 solvedProblemCount(int64) : 맞은 문제   
correctCount(int64) : 정답 횟수 (맞았습니다.)  
wrongCount(int64) : 오답 횟수 (틀렸습니다.)               
tiemOverCount(int64) : 시간초과 횟수 (시간초과)             
memoryOverCount(int64) : 메모리초과 횟수 (메모리 초과)   
overoutput(int64) : 출력초과 횟수 (출력초과)  
runtimeerrorCount(int64) : 런타임 에러 횟수 (런타임 에러)    
compileerrorCount(int64) : 컴파일 에러 횟수 (컴파일 에러)  
solvedProblemList : 맞은 문제(text : List)               
unSolvedProblemList : 시도했지만 맞지 못한 문제(text : List)

| 데이터 이름           | 데이터 설명             |데이터가 있는 위치         |
|--------------------|-----------------------|----------------------|
| solvedProblemCount | 맞은 문제               | u-solved tag뒤에 위치  |
| correctCount       | 정답 횟수 (맞았습니다.)    | u-result-4 tag뒤에 위치|
| wrongCount         | 오답 횟수 (틀렸습니다.)    | u-result-6 tag뒤에 위치|
| tiemOverCount      | 시간초과 횟수 (시간초과)    | u-result-7 tag뒤체 위치|
| memoryOverCount    | 메모리초과 횟수 (메모리 초과)| u-result-8 tag뒤에 위치|
| overoutputCount   | 출력초과 횟수(출력 초과)    | u-result-9 tag뒤에 위치|
| runtimeerrorCount  | 런타임에러 횟수(런타임 에러) | u-result-10 tag뒤에 위치|
| compileerrorCount  | 컴파일에러 횟수(컴파일 에러) | u-result-11 tag뒤에 위치|
| solvedProblemList  | 맞은 문제(text : List)   |panel-body에 problem list에 위치 |
| unSolvedProblemList|시도했지만 맞지 못한 문제(text : List)| panel-body problem list에 위치 |




In [3]:
def read_user_csv():
     return pd.read_csv("userData.csv")

In [33]:
# 백준에서 사용자의 정보를 크롤링 하는 함수
# 간격은 10분으로 크롤링을 진행한다. (ip ban을 방지)
def DataCrawl(userId):
    # BaseUrl인 BackJoon에 사용자 ID를 더해 크롤링 주소를 만든다.
    crawling_url = 'https://www.acmicpc.net/user/'
    crawling_url = crawling_url + userId
    
    # 정상적으로 크롤링을 하기 위해서 header를 삽입.
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(crawling_url, headers=headers)
    text = response.text
    status = response.status_code;
    if(status >= 400 | status < 500):
        Block_User.loc[len(Block_User)] = [userId];
    else:
        soup = bs(text, "html.parser")
        
        user_Detail = soup.find('table',{'class':'table table-hover'})
        for user_data in user_Detail:
            solved = user_data.find('span',{'id': 'u-solved'}).text
            correct = user_data.find('span',{'id': 'u-result-4'}).text
            wrong = user_data.find('span',{'id': 'u-result-6'}).text
            # 시간초과 횟수 항목이 있는지 확인   
            if (user_data.find('span',{'id': 'u-result-7'})):
                timeO = user_data.find('span',{'id': 'u-solved'}).text
            else:
                timeO = 0
            # 메모리 초과횟수 항목이 있는지 확인   
            if (user_data.find('span',{'id': 'u-result-8'})):
                memoryO = user_data.find('span',{'id': 'u-result-8'}).text
            else:
                memoryO = 0
            # 출력 에러 항목이 있는지 확인   
            if (user_data.find('span',{'id': 'u-result-9'})):
                overO = user_data.find('span',{'id': 'u-result-9'}).text
            else :
                overO = 0
            # 런타임 에러 항목이 있는지 확인   
            if (user_data.find('span',{'id': 'u-result-10'})):
                runtimeE = user_data.find('span',{'id': 'u-result-10'}).text
            else:
                runtimeE = 0
            # 컴파일 에러 항목이 있는지 확인
            if (user_data.find('span',{'id': 'u-result-11'})):
                compileE = user_data.find('span',{'id': 'u-result-11'}).text
            else:
                compileE = 0
            # Data Frame에 항목 추가
            User.loc[len(User)] = [userId, solved, correct, wrong, timeO, memoryO, overO, runtimeE, compileE]
        
            solved_Detail = soup.find_all('div',{'class':'problem-list'})
            for solved_list in solved_Detail[0]:
                if (solved_list.text == '\n' or solved_list.text == ' '):
                    temp = solved_list.text
                else:
                    solved = solved_list.text
                    User_solved.loc[len(User_solved)] = [userId, solved]
                
            for unsolved_list in solved_Detail[1]:
                if (unsolved_list.text == '\n' or unsolved_list.text == ' '):
                    temp = solved_list.text
                else:
                    unsolved = unsolved_list.text
                    User_unsolved.loc[len(User_unsolved)] = [userId, unsolved]

    print(userId)
    time.sleep(5)
    

In [34]:
User = pd.DataFrame (columns = 
    [
        "User",
        "SolvedProblemCount",
        "CorrectedCount",
        "WrongCount",
        "TimeOverCount",
        "MemoryOverCount",
        "OverOutputCount",
        "RuntimeErrorCount",
        "CompileErrorCount"
    ]
)

Block_User = pd.DataFrame(columns= 
    [
        "BlockUser"
    ]
)

User_solved= pd.DataFrame (columns = 
    [
        "User",
        "SolvedProblem"
    ]
)

User_unsolved = pd.DataFrame (columns = 
    [
        "User",
        "UnSolvedProblem"
    ]
)

In [35]:
# 사용자의 정보를 CSV에서 읽어온다.
user_csv = read_user_csv()
# 사용자의 이름에 대한 배열을 제작.
user_list = np.array(user_csv['handle'])

for user in (user_list):
    DataCrawl(user)

# Block User와 엉뚱한 User, 정상적인 사용자 테스트
#DataCrawl("xffff")
#ataCrawl("useridd")
#DataCrawl("asdasdsda")
#DataCrawl("arad4228")

404
<class 'int'>
xffff
404
<class 'int'>
useridd
404
<class 'int'>
asdasdsda
200
<class 'int'>
arad4228


In [36]:
User

,User,SolvedProblemCount,CorrectedCount,WrongCount,TimeOverCount,MemoryOverCount,OverOutputCount,RuntimeErrorCount,CompileErrorCount
0,arad4228,25,36,42,25,1,4,15,4


In [37]:
Block_User

,BlockUser
0,xffff
1,useridd
2,asdasdsda


In [ ]:
User_solved

In [ ]:
User_unsolved